# Instacart Market Basket Analysis

This is a kaggle competition wherein, we are provided with anonymised transaction records for a number of users/customers. The transactions contain a list of products that were purchased in each order. Based on the available information, our objective is to predict which of the products would be reordered in a given set of orders with the "test" label.

The data tables provided are:

1. Orders - This is a comprehensive list of all the summarized orders including Test, Train and Prior order_ids.
2. Order_prod_prior - THis is a subset of prior orders and contains product level details in each order.
3. Order_prod_train - This is a subset of train orders and contains product level details in each order.
4. Products - This is the product hierarchy table at product_id level
5. Aisles - this is is Aisle heirarchy and description table
6. Departments - This is the department level Description table

We are predicting the column "REORDERED" in the test dataset

### Import all the necessary libraries and the data

In [2]:
import pandas as pd
import numpy as np

products=pd.read_csv('products.csv')
aisles=pd.read_csv('aisles.csv')
departments=pd.read_csv('departments.csv')
orders=pd.read_csv('orders.csv')
order_prod_train=pd.read_csv('order_products__train.csv')
order_prod_prior=pd.read_csv('order_products__prior.csv')

### Creating order_prod_test

From the tables above we can create a dataframe with user_id and all the product_id that have preciously been purchased by them.
We would be predicting which of these product_id would occur in the next order

In [3]:
order_prod_test=orders[orders['eval_set']=='test'][['user_id','order_id']]
order_prod_test.columns=['user_id','order_id_test']
order_prod_test=order_prod_test.merge(orders[orders['eval_set']=='prior'],on='user_id')[['order_id','user_id','order_id_test']]
order_prod_test=order_prod_test.merge(order_prod_prior[['order_id','product_id']],on='order_id')[['user_id','order_id_test','product_id']]
order_prod_test.columns=['user_id','order_id','product_id']
order_prod_test=order_prod_test[['order_id','product_id']]
order_prod_test.head()

,order_id,product_id
0,2774568,9387
1,2774568,17668
2,2774568,15143
3,2774568,16797
4,2774568,39190


### Aggregating features for prior period

From the above tables we will join the order_prod_train table with the products table to append the aisle_id and department_id information into the detailed transaction table.

First step is to merge the order dataset with the prior dataset.
* Get the user_id for all the train records from orders table
* Get all the order_ids and product_ids for the above user_ids from the prior table
* summarize the above data to get number of prior orders, 

In [4]:
def compile_table(data_branch,df):
    # This relates to identifying all the user_id in the train period
    temp1=orders[orders['eval_set']==data_branch][['user_id','order_id']]
    # Get a list of user_id from the train_dataset
    temp2=temp1['user_id']

    # Merging the train user_ids with the orders in the prior timeframe to get order_ids
    # This creates a list of order_ids in the prior period for the user_id in the training dataset
    temp3=orders[orders['eval_set']=='prior'].merge(pd.DataFrame(temp2),on='user_id',how='inner')


    # Merging the above dataframe with the order_prod_prior to append a list of product_ids for each order
    temp3=temp3.merge(order_prod_prior[['order_id','product_id']],on='order_id',how='inner')
    temp3=temp3[['user_id','order_id','product_id','order_number','order_dow','order_hour_of_day','days_since_prior_order']]
    temp3=temp3.merge(products[['product_id','aisle_id','department_id']],on='product_id')


    # deleting the unnecessary tables to clear up some memory
    del temp1
    del temp2
    
    # Creating a list of unique combinations of user_id and product_id

    temp4=temp3[['user_id','order_id','product_id']].groupby(['user_id','product_id'],as_index=False).count()
    temp4.columns=['user_id','product_id','count_orders']
    temp4=temp4[['user_id','product_id']]
    temp4['temp']=1
#     temp4.head()
    
    # Create a unique list of order_dow to do a full outer join later
    temp5=pd.DataFrame(temp3[['order_dow','user_id']].groupby('order_dow',as_index=False).count()['order_dow'])
    temp5['temp']=1
    temp5
    
    # Merging the two dataframes above to create all possible combinations of user_id,product_id,order_dow
    temp6=pd.merge(temp4,temp5,on=['temp'])
    del temp6['temp']


    del temp5
    del temp4
#     temp6.head(14)
    
    # Creating a summary of user_id,prod_id,order_dow for number of orders

    temp7=temp3[['user_id','product_id','order_dow','order_id']].groupby(['user_id','product_id','order_dow'],as_index=False).count()
    temp7.columns=['user_id','product_id','order_dow','count_order_dow']

    # Merging the two tables tp create a dataframe that we can pivot
    temp8=temp6.merge(temp7,on=['user_id','product_id','order_dow'],how='left').fillna(0)

    del temp7
    del temp6
#     temp8.head(14)
    
    # Calculating the number of overall prders placed by user_id in the prior period
    temp9=temp3[['user_id','order_id','order_number']].groupby(['user_id','order_id'],as_index=False).count()
    temp9=temp9[['user_id','order_id']].groupby('user_id',as_index=False).count()
    temp9.columns=['user_id','count_orders']
#     temp9.head()
    
    # Calculating the percentage of orders per user in a dow
    temp10=temp8.merge(temp9,on='user_id',how='left')
    temp10['order_perc']=temp10['count_order_dow']/temp10['count_orders']
    del temp10['count_order_dow']
    del temp10['count_orders']
    del temp8
#     temp10.head()
    
    ## Lets create a index by hour for overall order history

    temp11=temp3[['user_id','order_hour_of_day','order_id','order_number']].groupby(['user_id','order_hour_of_day','order_id'],as_index=False).count()
    temp11=temp11[['user_id','order_hour_of_day','order_id']].groupby(['user_id','order_hour_of_day'],as_index=False).count()
    temp11.columns=['user_id','order_hour_of_day','count_orders_by_hour']
    temp11=temp11.merge(temp9,on='user_id',how='left')
    temp11['order_hour_perc']=temp11['count_orders_by_hour']/temp11['count_orders']
    del temp11['count_orders_by_hour']
    del temp11['count_orders']
#     temp11.head()
    
    # Order percenatge for each product at hour level
    temp12=temp3[['user_id','product_id','order_hour_of_day','order_id','order_number']].groupby(['user_id','product_id','order_hour_of_day','order_id'],as_index=False).count()
    temp12=temp12[['user_id','product_id','order_hour_of_day','order_id']].groupby(['user_id','product_id','order_hour_of_day'],as_index=False).count()
    temp12.columns=['user_id','product_id','order_hour_of_day','count_orders_by_hour']
    temp12=temp12.merge(temp9,on='user_id',how='left')
    temp12['order_prod_hour_perc']=temp12['count_orders_by_hour']/temp12['count_orders']
    del temp12['count_orders_by_hour']
    del temp12['count_orders']
    del temp9
#     temp12.head()

    # Calculating average reorder time at user level
    temp13=temp3[['user_id','order_id','days_since_prior_order']].fillna(0)
    temp13=temp13.groupby(['user_id','days_since_prior_order'],as_index=False).count()
    temp13=temp13[['user_id','days_since_prior_order']][temp13['days_since_prior_order']>0]
    temp13=temp13.groupby('user_id',as_index=False).mean()
    temp13.columns=['user_id','avg_preorder_days']
#     temp13.head()
    # Calculating the percentage of aisles shopped
    temp14=temp3[['user_id','aisle_id','product_id']].groupby(['user_id','aisle_id'],as_index=False).count()
    temp14=temp14[['user_id','aisle_id']].groupby('user_id',as_index=False).count()
    temp14.columns=['user_id','count_aisles']
    temp14['shopped_aisles']=temp14['count_aisles']/134
    del temp14['count_aisles']
#     temp14.head()

    # Calculating the percentage of depts shopped
    temp15=temp3[['user_id','department_id','product_id']].groupby(['user_id','department_id'],as_index=False).count()
    temp15=temp15[['user_id','department_id']].groupby('user_id',as_index=False).count()
    temp15.columns=['user_id','count_depts']
    temp15['shopped_depts']=temp15['count_depts']/21
    del temp15['count_depts']
#     temp15.head()
    
    # Calculating breadth of products shopped in each aisle
    temp16=temp3[['user_id','product_id','aisle_id','order_number']].groupby(['user_id','aisle_id','product_id'],as_index=False).count()
    temp16=temp16[['user_id','aisle_id','product_id']].groupby(['user_id','aisle_id'],as_index=False).count()
    temp16.columns=['user_id','aisle_id','count_prods']

    temp17=products[['aisle_id','product_id']].groupby('aisle_id',as_index=False).count()
    temp17.columns=['aisle_id','total_prods']

    temp16=temp16.merge(temp17,on='aisle_id',how='left')
    temp16['aisle_breadth']=temp16['count_prods']/temp16['total_prods']

    del temp16['total_prods']

    del temp17
#     temp16.head()
    
    # Combining all the tables above to create the final table
    temp_x=orders[orders['eval_set']==data_branch]
    arg=temp_x.merge(df,on='order_id')

    del arg['eval_set']
#     del arg['order_id']
    del arg['order_number']

    arg=arg.merge(temp12,on=['user_id','product_id','order_hour_of_day'],how='left')
    arg=arg.merge(temp11,on=['user_id','order_hour_of_day'],how='left')
    arg=arg.merge(temp10,on=['user_id','product_id','order_dow'],how='left')
    del arg['order_dow']
    del arg['order_hour_of_day']
    arg=arg.merge(temp13,on='user_id',how='left')
    arg['index_reorder']=arg['days_since_prior_order']/arg['avg_preorder_days']
    del arg['days_since_prior_order']
    del arg['avg_preorder_days']


    arg=arg.merge(temp15,on='user_id',how='left')
    arg=arg.merge(temp14,on='user_id',how='left')
    arg=arg.merge(products,on='product_id',how='left')
    arg=arg.merge(temp16,on=['user_id','aisle_id'])
    del temp15
    del temp11
    del temp12
    del temp13
    del temp14
    del temp16
    del temp10
    del arg['product_name']
    del arg['aisle_id']
    del arg['department_id']
    return arg
    


In [5]:
test_set=compile_table('test',order_prod_test)
test_set=test_set.fillna(0)
test_set.head()

,order_id,user_id,product_id,order_prod_hour_perc,order_hour_perc,order_perc,index_reorder,shopped_depts,shopped_aisles,count_prods,aisle_breadth
0,2774568,3,9387,0.083333,0.166667,0.0,0.785714,0.428571,0.119403,5,0.013089
1,2774568,3,15143,0.000000,0.166667,0.0,0.785714,0.428571,0.119403,5,0.013089
2,2774568,3,16797,0.083333,0.166667,0.0,0.785714,0.428571,0.119403,5,0.013089
3,2774568,3,47766,0.083333,0.166667,0.0,0.785714,0.428571,0.119403,5,0.013089
4,2774568,3,21137,0.000000,0.166667,0.0,0.785714,0.428571,0.119403,5,0.013089


In [6]:
train_set=compile_table('train',order_prod_train)
del train_set['add_to_cart_order']
train_set=train_set.fillna(0)
train_set.head()

,order_id,user_id,product_id,reordered,order_prod_hour_perc,order_hour_perc,order_perc,index_reorder,shopped_depts,shopped_aisles,count_prods,aisle_breadth
0,1187899,1,196,1,0.2,0.2,0.3,0.636364,0.333333,0.089552,2,0.004320
1,1187899,1,46149,1,0.1,0.2,0.1,0.636364,0.333333,0.089552,2,0.004320
2,1187899,1,25133,1,0.1,0.2,0.3,0.636364,0.333333,0.089552,1,0.001122
3,1187899,1,38928,1,0.1,0.2,0.1,0.636364,0.333333,0.089552,1,0.000975
4,1187899,1,26405,1,0.1,0.2,0.1,0.636364,0.333333,0.089552,1,0.003106


In [7]:
train_set=train_set.fillna(0)
train_set=train_set[['reordered','user_id','order_id','product_id','order_prod_hour_perc','order_hour_perc','order_perc','index_reorder','shopped_depts','shopped_aisles','count_prods','aisle_breadth']]
X=train_set.iloc[:,2:12].values
Y=train_set.iloc[:,0].values

In [8]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=0)


from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [9]:
X_train.shape

(813303, 10)

In [ ]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 10, kernel_initializer = 'glorot_uniform', activation = 'relu', input_dim = 10))
    classifier.add(Dense(units = 10, kernel_initializer = 'glorot_uniform', activation = 'relu'))
    classifier.add(Dense(units = 10, kernel_initializer = 'glorot_uniform', activation = 'relu'))
    classifier.add(Dense(units = 10, kernel_initializer = 'glorot_uniform', activation = 'relu'))
    classifier.add(Dense(units = 10, kernel_initializer = 'glorot_uniform', activation = 'relu'))
    classifier.add(Dense(units = 10, kernel_initializer = 'glorot_uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [5000, 1000],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, Y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/100
731972/731972 [==============================] - 1s - loss: 0.5956 - acc: 0.7077     
Epoch 2/100
731972/731972 [==============================] - 1s - loss: 0.4252 - acc: 0.7303     
Epoch 3/100
731972/731972 [==============================] - 1s - loss: 0.3998 - acc: 0.7418     
Epoch 4/100
731972/731972 [==============================] - 1s - loss: 0.3971 - acc: 0.7428     
Epoch 5/100
731972/731972 [==============================] - 1s - loss: 0.3957 - acc: 0.7443     
Epoch 6/100
731972/731972 [==============================] - 1s - loss: 0.3951 - acc: 0.7445     
Epoch 7/100
731972/731972 [==============================] - 1s - loss: 0.3945 - acc: 0.7450     
Epoch 8/100
731972/731972 [==============================] - 1s - loss: 0.3940 - acc: 0.7458     
Epoch 9/100
731972/731972 [==============================] - 1s - loss: 0.3941 - acc: 0.7452     
Epoch 10/100
731972/731972 [==============================] - 1s - loss: 0.3937 - acc: 0.7469     
Epoch 11/100
731972

731972/731972 [==============================] - 1s - loss: 0.3886 - acc: 0.7559     
Epoch 84/100
731972/731972 [==============================] - 1s - loss: 0.3886 - acc: 0.7558     
Epoch 85/100
731972/731972 [==============================] - 1s - loss: 0.3885 - acc: 0.7566     
Epoch 86/100
731972/731972 [==============================] - 1s - loss: 0.3884 - acc: 0.7565     
Epoch 87/100
731972/731972 [==============================] - 1s - loss: 0.3886 - acc: 0.7556     
Epoch 88/100
731972/731972 [==============================] - 1s - loss: 0.3886 - acc: 0.7559     
Epoch 89/100
731972/731972 [==============================] - 1s - loss: 0.3884 - acc: 0.7564     
Epoch 90/100
731972/731972 [==============================] - 1s - loss: 0.3883 - acc: 0.7568     
Epoch 91/100
731972/731972 [==============================] - 1s - loss: 0.3884 - acc: 0.7561     
Epoch 92/100
731972/731972 [==============================] - 1s - loss: 0.3885 - acc: 0.7563     
Epoch 93/100
731972/731

731972/731972 [==============================] - 1s - loss: 0.3894 - acc: 0.7551     
Epoch 66/100
731972/731972 [==============================] - 1s - loss: 0.3891 - acc: 0.7556     
Epoch 67/100
731972/731972 [==============================] - 1s - loss: 0.3898 - acc: 0.7541     
Epoch 68/100
731972/731972 [==============================] - 1s - loss: 0.3893 - acc: 0.7552     
Epoch 69/100
731972/731972 [==============================] - 1s - loss: 0.3892 - acc: 0.7554     
Epoch 70/100
731972/731972 [==============================] - 1s - loss: 0.3891 - acc: 0.7560     
Epoch 71/100
731972/731972 [==============================] - 1s - loss: 0.3893 - acc: 0.7555     
Epoch 72/100
731972/731972 [==============================] - 1s - loss: 0.3890 - acc: 0.7562     
Epoch 73/100
731972/731972 [==============================] - 1s - loss: 0.3889 - acc: 0.7560     
Epoch 74/100
731972/731972 [==============================] - 1s - loss: 0.3889 - acc: 0.7560     
Epoch 75/100
731972/731

731972/731972 [==============================] - 1s - loss: 0.3921 - acc: 0.7500     
Epoch 48/100
731972/731972 [==============================] - 1s - loss: 0.3922 - acc: 0.7497     
Epoch 49/100
731972/731972 [==============================] - 1s - loss: 0.3922 - acc: 0.7500     
Epoch 50/100
731972/731972 [==============================] - 1s - loss: 0.3919 - acc: 0.7506     
Epoch 51/100
731972/731972 [==============================] - 1s - loss: 0.3919 - acc: 0.7505     
Epoch 52/100
731972/731972 [==============================] - 1s - loss: 0.3918 - acc: 0.7508     
Epoch 53/100
731972/731972 [==============================] - 1s - loss: 0.3921 - acc: 0.7507     
Epoch 54/100
731972/731972 [==============================] - 1s - loss: 0.3920 - acc: 0.7506     
Epoch 55/100
731972/731972 [==============================] - 1s - loss: 0.3917 - acc: 0.7513     
Epoch 56/100
731972/731972 [==============================] - 1s - loss: 0.3916 - acc: 0.7516     
Epoch 57/100
731972/731

731973/731973 [==============================] - 1s - loss: 0.3918 - acc: 0.7510     
Epoch 30/100
731973/731973 [==============================] - 1s - loss: 0.3917 - acc: 0.7518     
Epoch 31/100
731973/731973 [==============================] - 1s - loss: 0.3921 - acc: 0.7505     
Epoch 32/100
731973/731973 [==============================] - 1s - loss: 0.3915 - acc: 0.7525     
Epoch 33/100
731973/731973 [==============================] - 1s - loss: 0.3911 - acc: 0.7527     
Epoch 34/100
731973/731973 [==============================] - 1s - loss: 0.3912 - acc: 0.7526     
Epoch 35/100
731973/731973 [==============================] - 1s - loss: 0.3913 - acc: 0.7522     
Epoch 36/100
731973/731973 [==============================] - 1s - loss: 0.3916 - acc: 0.7520     
Epoch 37/100
731973/731973 [==============================] - 1s - loss: 0.3910 - acc: 0.7532     
Epoch 38/100
731973/731973 [==============================] - 1s - loss: 0.3910 - acc: 0.7530     
Epoch 39/100
731973/731

...

731973/731973 [==============================] - 1s - loss: 0.3851 - acc: 0.7648     
Epoch 466/500
731973/731973 [==============================] - 1s - loss: 0.3853 - acc: 0.7642     
Epoch 467/500
731973/731973 [==============================] - 1s - loss: 0.3852 - acc: 0.7646     
Epoch 468/500
731973/731973 [==============================] - 1s - loss: 0.3852 - acc: 0.7641     
Epoch 469/500
731973/731973 [==============================] - 1s - loss: 0.3851 - acc: 0.7647     
Epoch 470/500
731973/731973 [==============================] - 1s - loss: 0.3851 - acc: 0.7648     
Epoch 471/500
731973/731973 [==============================] - 1s - loss: 0.3852 - acc: 0.7650     
Epoch 472/500
731973/731973 [==============================] - 1s - loss: 0.3852 - acc: 0.7644     
Epoch 473/500
731973/731973 [==============================] - 1s - loss: 0.3851 - acc: 0.7648     
Epoch 474/500
731973/731973 [==============================] - 1s - loss: 0.3851 - acc: 0.7646     
Epoch 475/500


731973/731973 [==============================] - 1s - loss: 0.3911 - acc: 0.7530     
Epoch 48/500
731973/731973 [==============================] - 1s - loss: 0.3910 - acc: 0.7532     
Epoch 49/500
731973/731973 [==============================] - 1s - loss: 0.3910 - acc: 0.7541     
Epoch 50/500
731973/731973 [==============================] - 1s - loss: 0.3906 - acc: 0.7546     
Epoch 51/500
731973/731973 [==============================] - 1s - loss: 0.3909 - acc: 0.7538     
Epoch 52/500
731973/731973 [==============================] - 1s - loss: 0.3906 - acc: 0.7548     
Epoch 53/500
731973/731973 [==============================] - 1s - loss: 0.3904 - acc: 0.7550     
Epoch 54/500
731973/731973 [==============================] - 1s - loss: 0.3905 - acc: 0.7549     
Epoch 55/500
731973/731973 [==============================] - 1s - loss: 0.3903 - acc: 0.7551     
Epoch 56/500
731973/731973 [==============================] - 1s - loss: 0.3901 - acc: 0.7555     
Epoch 57/500
731973/731

731973/731973 [==============================] - 1s - loss: 0.3893 - acc: 0.7558     
Epoch 130/500
731973/731973 [==============================] - 1s - loss: 0.3892 - acc: 0.7561     
Epoch 131/500
731973/731973 [==============================] - 1s - loss: 0.3892 - acc: 0.7560     
Epoch 132/500
731973/731973 [==============================] - 1s - loss: 0.3894 - acc: 0.7557     
Epoch 133/500
731973/731973 [==============================] - 1s - loss: 0.3891 - acc: 0.7564     
Epoch 134/500
731973/731973 [==============================] - 1s - loss: 0.3893 - acc: 0.7557     
Epoch 135/500
731973/731973 [==============================] - 1s - loss: 0.3892 - acc: 0.7560     
Epoch 136/500
731973/731973 [==============================] - 1s - loss: 0.3893 - acc: 0.7561     
Epoch 137/500
731973/731973 [==============================] - 1s - loss: 0.3892 - acc: 0.7558     
Epoch 138/500
731973/731973 [==============================] - 1s - loss: 0.3893 - acc: 0.7557     
Epoch 139/500


731973/731973 [==============================] - 1s - loss: 0.3868 - acc: 0.7600     
Epoch 212/500
731973/731973 [==============================] - 1s - loss: 0.3869 - acc: 0.7601     
Epoch 213/500
731973/731973 [==============================] - 1s - loss: 0.3869 - acc: 0.7603     
Epoch 214/500
731973/731973 [==============================] - 1s - loss: 0.3868 - acc: 0.7606     
Epoch 215/500
731973/731973 [==============================] - 1s - loss: 0.3868 - acc: 0.7604     
Epoch 216/500
731973/731973 [==============================] - 1s - loss: 0.3869 - acc: 0.7603     
Epoch 217/500
731973/731973 [==============================] - 1s - loss: 0.3869 - acc: 0.7602     
Epoch 218/500
731973/731973 [==============================] - 1s - loss: 0.3868 - acc: 0.7601     
Epoch 219/500
731973/731973 [==============================] - 1s - loss: 0.3868 - acc: 0.7604     
Epoch 220/500
731973/731973 [==============================] - 1s - loss: 0.3867 - acc: 0.7606     
Epoch 221/500


731973/731973 [==============================] - 1s - loss: 0.3864 - acc: 0.7623     
Epoch 294/500
731973/731973 [==============================] - 1s - loss: 0.3863 - acc: 0.7625     
Epoch 295/500
731973/731973 [==============================] - 1s - loss: 0.3864 - acc: 0.7621     
Epoch 296/500
731973/731973 [==============================] - 1s - loss: 0.3864 - acc: 0.7623     
Epoch 297/500
731973/731973 [==============================] - 1s - loss: 0.3863 - acc: 0.7626     
Epoch 298/500
731973/731973 [==============================] - 1s - loss: 0.3864 - acc: 0.7625     
Epoch 299/500
731973/731973 [==============================] - 1s - loss: 0.3863 - acc: 0.7626     
Epoch 300/500
731973/731973 [==============================] - 1s - loss: 0.3863 - acc: 0.7628     
Epoch 301/500
731973/731973 [==============================] - 1s - loss: 0.3864 - acc: 0.7622     
Epoch 302/500
731973/731973 [==============================] - 1s - loss: 0.3864 - acc: 0.7624     
Epoch 303/500


In [11]:
# Importing the keras libraries to create out Neural Networks

import keras
from keras.models import Sequential
from keras.layers import Dense

# Defining the initial object as a sequential model with more than one layers
clf=Sequential()

# Adding the first hidden layer
# output_dim is the average of input dims=7 and output dims=1.
# init is the function to initialize the weights. uniform is the most simple way to add a close to 0 value for weights 
# Choosing the activation function to be relu which corresponds to rectifier function 
clf.add(Dense(units=10,kernel_initializer="glorot_uniform",activation='relu',input_dim=10))

#Adding the second hidden layer
clf.add(Dense(units=10,kernel_initializer="glorot_uniform",activation='relu'))

#Adding the third hidden layer
clf.add(Dense(units=10,kernel_initializer="glorot_uniform",activation='relu'))


#Adding the fourth hidden layer
clf.add(Dense(units=10,kernel_initializer="glorot_uniform",activation='relu'))


#Adding the fifth hidden layer
clf.add(Dense(units=10,kernel_initializer="glorot_uniform",activation='relu'))


# Adding the final output layer
clf.add(Dense(units=1,kernel_initializer="glorot_uniform",activation='sigmoid'))

# Compiling using the ADAM optimizer for Stachastic Gradient Descent
clf.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# Lets fit this for 100 epochs and a batch_size of 10
clf.fit(X_train,Y_train,batch_size=1000,epochs=100)


Epoch 1/100
813303/813303 [==============================] - 2s - loss: 0.4989 - acc: 0.7186     
Epoch 2/100
813303/813303 [==============================] - 1s - loss: 0.3982 - acc: 0.7404     
Epoch 3/100
813303/813303 [==============================] - 1s - loss: 0.3956 - acc: 0.7429     
Epoch 4/100
813303/813303 [==============================] - 1s - loss: 0.3945 - acc: 0.7441     
Epoch 5/100
813303/813303 [==============================] - 1s - loss: 0.3937 - acc: 0.7451     
Epoch 6/100
813303/813303 [==============================] - 1s - loss: 0.3930 - acc: 0.7471     
Epoch 7/100
813303/813303 [==============================] - 1s - loss: 0.3923 - acc: 0.7494     
Epoch 8/100
813303/813303 [==============================] - 1s - loss: 0.3917 - acc: 0.7516     
Epoch 9/100
813303/813303 [==============================] - 1s - loss: 0.3912 - acc: 0.7522     
Epoch 10/100
813303/813303 [==============================] - 1s - loss: 0.3909 - acc: 0.7531     
Epoch 11/100
813303

813303/813303 [==============================] - 1s - loss: 0.3863 - acc: 0.7614     
Epoch 84/100
813303/813303 [==============================] - 1s - loss: 0.3862 - acc: 0.7617     
Epoch 85/100
813303/813303 [==============================] - 1s - loss: 0.3863 - acc: 0.7613     
Epoch 86/100
813303/813303 [==============================] - 1s - loss: 0.3863 - acc: 0.7618     
Epoch 87/100
813303/813303 [==============================] - 1s - loss: 0.3863 - acc: 0.7619     
Epoch 88/100
813303/813303 [==============================] - 1s - loss: 0.3862 - acc: 0.7621     
Epoch 89/100
813303/813303 [==============================] - 1s - loss: 0.3863 - acc: 0.7615     
Epoch 90/100
813303/813303 [==============================] - 1s - loss: 0.3861 - acc: 0.7621     
Epoch 91/100
813303/813303 [==============================] - 1s - loss: 0.3862 - acc: 0.7622     
Epoch 92/100
813303/813303 [==============================] - 1s - loss: 0.3862 - acc: 0.7621     
Epoch 93/100
813303/813

In [ ]:
Y_pred=clf.predict(X_test)
Y_pred=(Y_pred>0.5)

# Calculating the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,Y_pred)
cm

In [ ]:
test_set=test_set.fillna(0)
test_set=test_set[['order_id','product_id','order_prod_hour_perc','order_hour_perc','order_perc','index_reorder','shopped_depts','shopped_aisles','count_prods','aisle_breadth']]
X=test_set.iloc[:,0:10].values
X=sc.fit_transform(X)

Y_final=clf.predict(X)

submission_file=pd.DataFrame(Y_final)
submission_file['order_id']=test_set['order_id']
submission_file['product_id']=test_set['product_id']
submission_file.columns=['prob','order_id','product_id']
submission_file=submission_file[submission_file['prob']>0.5]
submission_file.head()